# LAB GenAI - LLMs - OpenAI Assistant

In [3]:
from IPython.core.display import display, Markdown

/var/folders/g2/dsm99_hd0c9_m_6qhbpwpvl80000gn/T/ipykernel_13108/3048415430.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, Markdown


In [4]:
# Import dotenv to load env file
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv(dotenv_path='openaikey.env')

# Get the API key from the environment variable
openai_api_key = os.getenv("OPENAI_API_KEY")

## Exercise 1

Create an assistant to answer a topic of your choosing:
 - Upload a file of your interest
 - Add Instructions to the prompt
 - Use the assistant in Playground mode

 https://platform.openai.com/playground/assistants

In [7]:
import pandas as pd
from openai import OpenAI

In [8]:
client = OpenAI(api_key=openai_api_key)

assistant = client.beta.assistants.create(
    name='Avatar Wilderness Guide',
    instructions='You are a Pandora Wilderness Guide based off of the James Cameron Avatar movies that gives a user useful information about the Pandoran wilderness.',
    model='gpt-4o-mini',
    tools=[{'type': 'file_search'}],
)

In [9]:
# Create a vector store caled "Pandora Documents"
vector_store = client.beta.vector_stores.create(name='Pandora Documents')

In [17]:
# Upload files to OpenAI
file_paths = ['PDFTechlib-Brochure.pdf']
file_streams = [open(path, 'rb') for path in file_paths]

In [19]:
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

In [20]:
# Print files
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [23]:
assistant = client.beta.assistants.update(
    assistant_id = assistant.id,
    tool_resources={'file_search' : {'vector_store_ids' : [vector_store.id]}},
)

## Exercise 2

Talk to your assistant via the API

https://platform.openai.com/docs/assistants/overview

In [25]:
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
    messages = [
        {
            'role' : 'user',
            'content' : 'What generate the tool?'
        }
    ]
)

run = client.beta.threads.runs.create_and_poll(
    thread_id = thread.id, assistant_id = assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
message_content = messages[0].content[0].text
display(Markdown(message_content.value))

The tool allows me to search through the documents you have uploaded. I can issue queries to extract specific information from those documents. If you have a specific question or topic in mind related to the content of your files, please let me know, and I can help find that information for you!

## Exercise 3

Create an assistant that will call a weather API, given the user's answer and return the proper answer.

See the documentation of the weather API here: https://open-meteo.com/en/docs

In [27]:
import requests

def get_weather_forecast(latitude, longitude):
    base_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m"
    }
    response = requests.get(base_url, params=params)
    return response.json()

# Example usage:
forecast = get_weather_forecast(52.52, 13.41)
print(forecast)

{'latitude': 52.52, 'longitude': 13.419998, 'generationtime_ms': 0.05555152893066406, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 38.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'}, 'hourly': {'time': ['2025-02-14T00:00', '2025-02-14T01:00', '2025-02-14T02:00', '2025-02-14T03:00', '2025-02-14T04:00', '2025-02-14T05:00', '2025-02-14T06:00', '2025-02-14T07:00', '2025-02-14T08:00', '2025-02-14T09:00', '2025-02-14T10:00', '2025-02-14T11:00', '2025-02-14T12:00', '2025-02-14T13:00', '2025-02-14T14:00', '2025-02-14T15:00', '2025-02-14T16:00', '2025-02-14T17:00', '2025-02-14T18:00', '2025-02-14T19:00', '2025-02-14T20:00', '2025-02-14T21:00', '2025-02-14T22:00', '2025-02-14T23:00', '2025-02-15T00:00', '2025-02-15T01:00', '2025-02-15T02:00', '2025-02-15T03:00', '2025-02-15T04:00', '2025-02-15T05:00', '2025-02-15T06:00', '2025-02-15T07:00', '2025-02-15T08:00', '2025-02-15T09:00', '2025-02-15T10:00', '2025-02-15T11:00', '2025-02-15T12:00'

### If you want to, there is a hint here:

OpenAI Chatbots / Assistants have a way to respond in json format. 

Explore the function calling functionality